# Plugin Interface

> Abstract base class defining the generic plugin interface

In [ ]:
#| default_exp core.interface

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import ABC, abstractmethod
from typing import Any, Dict, Optional, Generator, Protocol, runtime_checkable

## FileBackedDTO Protocol

The `FileBackedDTO` protocol defines objects that can serialize themselves to disk for zero-copy transfer between Host and Worker processes. When the Proxy detects an argument implementing this protocol, it calls `to_temp_file()` and sends the file path instead of the data.

In [ ]:
#| export
@runtime_checkable
class FileBackedDTO(Protocol):
    """Protocol for Data Transfer Objects that serialize to disk for zero-copy transfer."""
    
    def to_temp_file(self) -> str: # Absolute path to the temporary file
        """Save the data to a temporary file and return the absolute path."""
        ...

## PluginInterface

The `PluginInterface` is an abstract base class that defines the contract all plugins must implement. This interface works for both:

- **Concrete Plugins**: Running in Worker processes (implement the actual logic)
- **Remote Proxies**: Running in Host process (forward calls over HTTP)

The interface is domain-agnostic. Domain-specific plugin systems (e.g., transcription, vision) subclass this to add their specific methods and DTOs.

In [ ]:
#| export
class PluginInterface(ABC):
    """Abstract base class for all plugins (both local workers and remote proxies)."""

    @property
    @abstractmethod
    def name(self) -> str: # Unique identifier for this plugin
        """Unique plugin identifier."""
        ...

    @property
    @abstractmethod
    def version(self) -> str: # Semantic version string (e.g., "1.0.0")
        """Plugin version."""
        ...

    @abstractmethod
    def initialize(
        self,
        config: Optional[Dict[str, Any]] = None # Configuration dictionary
    ) -> None:
        """Initialize or re-configure the plugin."""
        ...

    @abstractmethod
    def execute(
        self,
        *args,
        **kwargs
    ) -> Any: # Plugin-specific output
        """Execute the plugin's main functionality."""
        ...

    def execute_stream(
        self,
        *args,
        **kwargs
    ) -> Generator[Any, None, None]: # Yields partial results
        """Stream execution results chunk by chunk."""
        # Default: yield single result from execute()
        yield self.execute(*args, **kwargs)

    @abstractmethod
    def get_config_schema(self) -> Dict[str, Any]: # JSON Schema for configuration
        """Return JSON Schema describing the plugin's configuration options."""
        ...

    @abstractmethod
    def get_current_config(self) -> Dict[str, Any]: # Current configuration values
        """Return the current configuration state as a dictionary."""
        ...

    @abstractmethod
    def cleanup(self) -> None:
        """Clean up resources when plugin is unloaded."""
        ...

The interface provides:

- **Identity**: `name` and `version` properties for plugin identification
- **Lifecycle**: `initialize()` for configuration and `cleanup()` for resource release
- **Execution**: `execute()` for main logic, `execute_stream()` for streaming results
- **Configuration**: `get_config_schema()` returns JSON Schema, `get_current_config()` returns current values

The default `execute_stream()` implementation yields a single result from `execute()`. Plugins can override this for true streaming where partial results are yielded as they become available.

## Example: Implementing a Plugin

Here's a complete example showing how to implement a concrete plugin:

In [ ]:
from dataclasses import dataclass, field, asdict

@dataclass
class ExampleConfig:
    """Configuration for the example plugin."""
    mode: str = "balanced"
    threshold: float = 0.5
    max_workers: int = 4

class ExamplePlugin(PluginInterface):
    """A simple example plugin implementation."""

    def __init__(self):
        self._config: ExampleConfig = ExampleConfig()
        self._resource: Optional[str] = None

    @property
    def name(self) -> str:
        return "example-plugin"
    
    @property
    def version(self) -> str:
        return "1.0.0"
    
    def initialize(self, config: Optional[Dict[str, Any]] = None) -> None:
        """Initialize or re-configure the plugin."""
        if config is None:
            config = {}
        
        # Merge with defaults
        current = asdict(self._config)
        current.update(config)
        self._config = ExampleConfig(**current)
        
        # Initialize resources based on config
        self._resource = f"Resource-{self._config.mode}"

    def execute(self, input_data: str, **kwargs) -> str:
        """Process input data."""
        return f"Processed '{input_data}' using {self._resource}"

    def get_config_schema(self) -> Dict[str, Any]:
        """Return JSON Schema for configuration."""
        return {
            "type": "object",
            "properties": {
                "mode": {
                    "type": "string",
                    "enum": ["fast", "balanced", "quality"],
                    "default": "balanced"
                },
                "threshold": {
                    "type": "number",
                    "minimum": 0.0,
                    "maximum": 1.0,
                    "default": 0.5
                },
                "max_workers": {
                    "type": "integer",
                    "minimum": 1,
                    "maximum": 16,
                    "default": 4
                }
            }
        }

    def get_current_config(self) -> Dict[str, Any]:
        """Return current configuration."""
        return asdict(self._config)

    def cleanup(self) -> None:
        """Clean up resources."""
        self._resource = None

In [ ]:
# Test the example plugin
plugin = ExamplePlugin()
plugin.initialize()

print(f"Plugin: {plugin.name} v{plugin.version}")
print(f"\nConfig Schema:")
print(plugin.get_config_schema())
print(f"\nCurrent Config:")
print(plugin.get_current_config())

Plugin: example-plugin v1.0.0

Config Schema:
{'type': 'object', 'properties': {'mode': {'type': 'string', 'enum': ['fast', 'balanced', 'quality'], 'default': 'balanced'}, 'threshold': {'type': 'number', 'minimum': 0.0, 'maximum': 1.0, 'default': 0.5}, 'max_workers': {'type': 'integer', 'minimum': 1, 'maximum': 16, 'default': 4}}}

Current Config:
{'mode': 'balanced', 'threshold': 0.5, 'max_workers': 4}


In [ ]:
# Test execution
result = plugin.execute("sample_data")
print(f"Result: {result}")

# Test re-initialization with new config
plugin.initialize({"mode": "quality", "threshold": 0.8})
print(f"\nAfter re-init config: {plugin.get_current_config()}")
result = plugin.execute("more_data")
print(f"Result: {result}")

# Test cleanup
plugin.cleanup()
print(f"\nAfter cleanup, resource is cleared")

Result: Processed 'sample_data' using Resource-balanced

After re-init config: {'mode': 'quality', 'threshold': 0.8, 'max_workers': 4}
Result: Processed 'more_data' using Resource-quality

After cleanup, resource is cleared


In [ ]:
# Test streaming (default implementation yields single result)
plugin.initialize({"mode": "balanced"})

print("Streaming execution:")
for chunk in plugin.execute_stream("stream_data"):
    print(f"  Chunk: {chunk}")

Streaming execution:
  Chunk: Processed 'stream_data' using Resource-balanced


In [ ]:
# Test FileBackedDTO protocol detection
import tempfile

class MockAudioData:
    """Example class implementing FileBackedDTO."""
    
    def __init__(self, data: bytes):
        self._data = data
    
    def to_temp_file(self) -> str:
        """Save to temp file and return path."""
        with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as f:
            f.write(self._data)
            return f.name

# Check if it implements the protocol
audio = MockAudioData(b"fake audio data")
print(f"MockAudioData implements FileBackedDTO: {isinstance(audio, FileBackedDTO)}")
print(f"Temp file path: {audio.to_temp_file()}")

# A regular string does not implement the protocol
print(f"str implements FileBackedDTO: {isinstance('hello', FileBackedDTO)}")

MockAudioData implements FileBackedDTO: True
Temp file path: /tmp/tmpghzo_76s.wav
str implements FileBackedDTO: False


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()